In [10]:
!pip install fastapi uvicorn pandas scikit-learn joblib python-multipart


<frozen posixpath>:82: RuntimeWarning: coroutine 'Server.serve' was never awaited


In [11]:
from fastapi import FastAPI, UploadFile, File
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
import uvicorn
import random


In [12]:
app = FastAPI(title="ModelSphere API")


In [13]:
@app.post("/api/login")
def login(data: dict):
    email = data.get("email")
    password = data.get("password")

    if email and len(password) > 5:
        return {
            "token": "real-jwt-token",
            "user": {
                "name": "Dr. Alex Rivera",
                "email": email,
                "avatar": "AR"
            }
        }
    return {"error": "Invalid credentials"}


In [14]:
@app.post("/api/upload")
async def upload(file: UploadFile = File(...)):
    content = await file.read()
    path = f"uploaded_{file.filename}"

    with open(path, "wb") as f:
        f.write(content)

    df = pd.read_csv(path)

    return {
        "filename": file.filename,
        "rows": len(df)
    }


In [15]:
@app.post("/api/train")
def train(data: dict):
    file_path = "uploaded_iris.csv"  # example

    df = pd.read_csv(file_path)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)

    cm = confusion_matrix(y_test, preds).tolist()

    report_raw = classification_report(y_test, preds, output_dict=True)
    report = []
    for cls in list(report_raw.keys())[:-3]:
        report.append({
            "class": cls,
            "precision": f"{report_raw[cls]['precision']:.2f}",
            "recall": f"{report_raw[cls]['recall']:.2f}",
            "f1": f"{report_raw[cls]['f1-score']:.2f}"
        })

    return {
        "accuracy": f"{acc*100:.1f}%",
        "status": "Deployed",
        "privacyScore": "Medium",
        "confusionMatrix": cm,
        "classes": list(set(y)),
        "report": report
    }


In [16]:
@app.get("/api/metrics")
def metrics():
    return {
        "requests": random.randint(10000, 20000),
        "cpu": f"{random.randint(20,60)}%",
        "memory": "2.1 GB",
        "privacy_audits": random.randint(5,20)
    }


In [18]:
import uvicorn
import nest_asyncio
import threading

# Apply nest_asyncio globally to patch asyncio for nested event loops
nest_asyncio.apply()

def start_uvicorn_server():
    """Function to run uvicorn in a separate thread."""
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start uvicorn in a new daemon thread
uvicorn_thread = threading.Thread(target=start_uvicorn_server, daemon=True)
uvicorn_thread.start()

print("FastAPI app is running in the background on http://0.0.0.0:8000")
print("You can access it via a public URL if you use ngrok or similar services to expose the port.")

FastAPI app is running in the background on http://0.0.0.0:8000
You can access it via a public URL if you use ngrok or similar services to expose the port.


INFO:     Started server process [174]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
